In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../../LIB/')
from env import ENV
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [2]:
X = pd.read_csv(ENV.installments_payments_ori.value)

In [3]:
class ordinal_encoder:
    def __init__(self,fillna='NAN_FILL'):
        self.fillna = fillna
    
    def fit(self,sr,NA_VALUE=None,realNA2Nega1=True):
        """
        spycify the value which has already been encoded. make it to -1 after encoding
        """
        self.NA_VALUE=NA_VALUE
        order = sr.fillna(self.fillna).value_counts()
        if self.NA_VALUE is not None:
            order[NA_VALUE] = 9999999

        if realNA2Nega1:
            order[self.fillna] = 99999999
        order = order.sort_values(ascending=True)


        self.mapping = pd.Series(index=order.index.values, data=list(range(1,len(order)+1)))
        if self.NA_VALUE is not None:
            self.mapping[NA_VALUE] = -1

        if realNA2Nega1:
            self.mapping[self.fillna] = -1
    
    def transform(self,sr):
        return sr.fillna(self.fillna).map(self.mapping)
        

In [4]:
def calculate_na(ser):
    return np.sum(ser.isnull())

def view_hist(ser):
    plt.hist(ser, range=(min(ser.values), max(ser.values)))
    
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)


In [5]:
p = scan_nan_portion(X)
p = p.sort_values()
print(p.describe())
oe = ordinal_encoder()

count    8.000000
mean     0.000053
std      0.000099
min      0.000000
25%      0.000000
50%      0.000000
75%      0.000053
max      0.000214
dtype: float64


# Data Clean

## 0 - 20

In [6]:
category_20 = []
failed = []
for each in p.iloc[:].index.values:
    
    print(X[each].describe())
    print('----')
    print(X[each].dtypes)
    num_values = len(X[each].value_counts())
    if num_values < 10:
        category_20.append(each)
        try:
            X[each] = X[each].astype('int')
        except ValueError:
            print('{} converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
            failed.append(each)
    print('value counts: {}'.format(num_values))
    print('NA percentage: {}'.format(p[each]))
    print('========= End ==================')
print(failed)
# category_20.remove()

count    1.360540e+07
mean     1.903365e+06
std      5.362029e+05
min      1.000001e+06
25%      1.434191e+06
50%      1.896520e+06
75%      2.369094e+06
max      2.843499e+06
Name: SK_ID_PREV, dtype: float64
----
int64
value counts: 997752
NA percentage: 0.0
========= End ==================
count    1.360540e+07
mean     2.784449e+05
std      1.027183e+05
min      1.000010e+05
25%      1.896390e+05
50%      2.786850e+05
75%      3.675300e+05
max      4.562550e+05
Name: SK_ID_CURR, dtype: float64
----
int64
value counts: 339587
NA percentage: 0.0
========= End ==================
count    1.360540e+07
mean     8.566373e-01
std      1.035216e+00
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.780000e+02
Name: NUM_INSTALMENT_VERSION, dtype: float64
----
float64
value counts: 65
NA percentage: 0.0
========= End ==================
count    1.360540e+07
mean     1.887090e+01
std      2.666407e+01
min      1.000000e+00
25%      4.000000e+00
5

In [7]:
col = 'NUM_INSTALMENT_VERSION'
X[col].value_counts()

1.0      8485004
0.0      4082498
2.0       620283
3.0       237063
4.0        55274
5.0        48404
6.0        17092
7.0        16771
9.0         8359
8.0         7814
10.0        4637
11.0        4342
13.0        2951
12.0        2863
15.0        1917
14.0        1906
16.0        1283
17.0        1249
18.0         883
19.0         816
20.0         615
21.0         589
22.0         426
23.0         373
24.0         291
25.0         268
26.0         211
27.0         177
29.0         163
28.0         145
          ...   
35.0          46
39.0          42
36.0          36
37.0          33
38.0          29
40.0          19
43.0          19
41.0          18
42.0          18
44.0          13
61.0           8
45.0           8
72.0           7
46.0           5
47.0           4
48.0           3
49.0           3
50.0           3
52.0           3
51.0           2
53.0           1
54.0           1
55.0           1
56.0           1
57.0           1
58.0           1
59.0           1
178.0         

### Ordinal Encoding

In [8]:
# col = 'NAME_CONTRACT_STATUS'

# oe.fit(X[col],NA_VALUE='XNA')
# X[col] = oe.transform(X[col])
# category_20.append(col)
###############################################



### FillNa

In [9]:
###############################################

col = 'DAYS_ENTRY_PAYMENT'
X[col] = X[col].fillna(X[col].mean())

###############################################

col = 'AMT_PAYMENT'
X[col] = X[col].fillna(X[col].mean())



# Saving

In [10]:
X.to_pickle(ENV.installments_payments_clean.value)

In [11]:
np.sum(X.isnull())

SK_ID_PREV                0
SK_ID_CURR                0
NUM_INSTALMENT_VERSION    0
NUM_INSTALMENT_NUMBER     0
DAYS_INSTALMENT           0
DAYS_ENTRY_PAYMENT        0
AMT_INSTALMENT            0
AMT_PAYMENT               0
dtype: int64

In [12]:
categori_col = list(set(category_20))

In [14]:
import pickle
categori_col_exist = pickle.load(open(ENV.clean_categorical_col.value,'rb'))
categori_col = list(set(categori_col + categori_col_exist))
pickle.dump(categori_col,open(ENV.clean_categorical_col.value,'wb'))